<font color="orange" size="2">
Exercicis de Web Scraping: web scraping i documentar dades recollides amb web scraping.</font>

# EX01. Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després.

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

# BeautifulSoup

Extraeremos los datos de `cita`, `autor` y `palabras clave` de http://quotes.toscrape.com, adecuada para BeautifulSoup al ser estática ...

In [64]:
import requests
from bs4 import BeautifulSoup

In [65]:
URL = "http://quotes.toscrape.com/"
page = requests.get(URL)

In [66]:
soup = BeautifulSoup(page.content, "html.parser")

In [67]:
results = soup.find_all(class_="quote")

... para todas las páginas, mediante una función sobre la que vamos a iterar para cada una ...

In [68]:
import pandas as pd

In [69]:
import datetime
import pytz

Importamos también las librerías que vamos a utilizar para crear el dataframe en el que guardar los datos y para recoger la fecha y hora de la consulta.

El dataframe será `citas`, con las columnas: `cita`, `autor` y `tags`:

In [70]:
citas = pd.DataFrame(columns=['cita', 'autor', 'tags'])

In [71]:
def extraer(n, results):
  i = 0
  for result in results:
      texto = result.find("span", class_="text")
      autor = result.find("small", class_="author")
      tags = result.find_all("a", class_="tag")
      fila = (n-1)*10+i
      citas.loc[fila, 'cita'] = texto.text.strip()
      citas.loc[fila, 'autor'] = autor.text.strip()
      tag_lista = []
      for tag in tags:
        tag_lista.append(tag.text.strip())
      citas.loc[fila, 'tags'] = ' '.join(tag_lista)
      i += 1

Llamamos a la función pasándole como parámetros el número de página y el contenido:

In [72]:
fecha_hora = datetime.datetime.utcnow().astimezone(pytz.timezone('Europe/Madrid')).strftime('%Y-%m-%d %H:%M:%S')
print('Guardado en citas ('+fecha_hora+')')

for n in range(1,11):
  URL = "http://quotes.toscrape.com/page/"+str(n)
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  results = soup.find_all(class_="quote")
  extraer(n, results)

Guardado en citas (2024-02-02 12:53:38)


In [85]:
citas

cita               autor  \
0   “The world as we have created it is a process ...     Albert Einstein   
1   “It is our choices, Harry, that show what we t...        J.K. Rowling   
2   “There are only two ways to live your life. On...     Albert Einstein   
3   “The person, be it gentleman or lady, who has ...         Jane Austen   
4   “Imperfection is beauty, madness is genius and...      Marilyn Monroe   
..                                                ...                 ...   
95  “You never really understand a person until yo...          Harper Lee   
96  “You have to write the book that wants to be w...   Madeleine L'Engle   
97  “Never tell the truth to people who are not wo...          Mark Twain   
98        “A person's a person, no matter how small.”           Dr. Seuss   
99  “... a mind needs books as a sword needs a whe...  George R.R. Martin   

                                                 tags  
0                 change deep-thoughts thinking world  
1                                   abilities choices  
2            inspirational life live miracle miracles  
3                       aliteracy books classic humor  
4                           be-yourself inspirational  
..                                                ...  
95                                better-life-empathy  
96  books children difficult grown-ups write write...  
97                                              truth  
98                                      inspirational  
99                                         books mind  

[100 rows x 3 columns]

... y obtenemos `cita`, `autor` y `palabras clave` para cada cita de cada página.

# Selenium

De manera similar a como hemos hecho anteriormente, vamos extraer el contenido de una web pero ahora dinámica y con la biblioteca Selenium: https://es.wikipedia.org/wiki/Canis_familiaris#

In [ ]:
pip install selenium

En primer lugar instalamos selenium y nos aseguramos de tener actualizada la última versión.

In [76]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

In [77]:
opts = Options()
opts.add_argument('--headless')
opts.headless = True
assert opts.headless
browser = Firefox(options=opts)

In [78]:
browser.get('https://es.wikipedia.org/wiki/Canis_familiaris#')

En segundo lugar importamos todos los módulos necesarios para poder hacer las llamadas a la web. Trabajamos con Firefox sin cabecera, para hacerlo compatible con Google Colab, pero también con la terminal ...

In [79]:
import re

In [80]:
wiki_perro = pd.DataFrame(columns=['texto', 'etiqueta'])

In [81]:
elementos = browser.find_elements(By.XPATH, "//h2 | //h3 | //h4 | //p")
i = 0

fecha_hora = datetime.datetime.utcnow().astimezone(pytz.timezone('Europe/Madrid')).strftime('%Y-%m-%d %H:%M:%S')
print('Guardado en wiki_perro ('+fecha_hora+')')

for elemento in elementos:
  if elemento.tag_name != 'sup':
    texto = re.sub(r'\.\d{1,8}', '.', elemento.text)
    texto2 = re.sub(r'\,\d{1,8}', ',', texto)
    texto3 = re.sub(r'1758', '', texto2)
    wiki_perro.loc[i, 'texto'] = texto3
    wiki_perro.loc[i, 'etiqueta'] = elemento.tag_name
    i += 1

browser.quit()

Guardado en wiki_perro (2024-02-02 12:54:33)


In [84]:
wiki_perro

texto etiqueta
0                                           Contenidos       h2
1    Canis familiaris Linnaeus \nCanis familiaris d...        p
2    El perro (Canis familiaris o Canis lupus famil...        p
3    Su tamaño (o talla), su forma y su pelaje es m...        p
4    El perro doméstico proviene de un grupo ancest...        p
..                                                 ...      ...
116  Los perros de guardia ayudan a proteger la pro...        p
117  Finalmente, los perros de guerra se usan en la...        p
118                                      Véase también       h2
119                                        Referencias       h2
120                                   Enlaces externos       h2

[121 rows x 2 columns]

Obtenemos el contenido textual y lo guardamos en el dataframe `contenido`, añadiendo para cada registro el tipo de etiqueta al que corresponde. Notar que el orden en que se guardan los elementos es el mismo en el que aparecen en la página, lo que nos permite reconstruir el contenido textual íntegramente. Finalmente cerramos la sesión.

# EX02. Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

A manera d'exemple del que es demana pots consultar aquest enllaç:

 https://www.kaggle.com/datasets/vivovinco/20212022-football-team-stats

Subido como archivo adjunto.

# EX03. Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després.

# Selenium:

Extraeremos las últimas noticias sobre tecnología de la web de Xataka:

In [86]:
browser = Firefox(options=opts)

Abrimos de nuevo sesión en el navegador y cargamos la web ...

In [87]:
noticias = pd.DataFrame(columns=['contenido', 'enlace'])
fila = 0

for n in range(00, 181, 20):
  URL = "https://api.scrapingdog.com/scrape?api_key=65bb87fb6f171f73bf007799&url=https://www.xataka.com/tag/tecnologia/record/"+str(n)+"&dynamic=false"
  browser.get(URL)
  articulos = browser.find_elements(By.XPATH, "//article")
  for articulo in articulos:
    contenido = articulo.text
    elemento_enlace = articulo.find_element(By.TAG_NAME, 'a')
    enlace = elemento_enlace.get_attribute('href')
    enlace2 = enlace.replace('api.scrapingdog.com', 'xataka.com')
    noticias.loc[fila, 'contenido'] = contenido
    noticias.loc[fila, 'enlace'] = enlace2
    fila += 1

fecha_hora = datetime.datetime.utcnow().astimezone(pytz.timezone('Europe/Madrid')).strftime('%Y-%m-%d %H:%M:%S')
print('Guardado en noticias ('+fecha_hora+')')

Guardado en noticias (2024-02-02 12:58:11)


In [88]:
noticias

contenido  \
0    ASML tiene el mejor equipo de litografía del m...   
1    Estos son los tres países y las cinco empresas...   
2    Llevamos meses esperando la gran jugada de App...   
3    EEUU no quiere perder la guerra de los chips. ...   
4    La IA ha disparado las previsiones de ventas d...   
..                                                 ...   
195  Una alianza de 30 países, entre ellos España, ...   
196  Se llama Aitana, es influencer y triunfa en In...   
197  Windows 10 se quedará sin soporte en dos años:...   
198  Samsung ha pisado el acelerador: esta es la te...   
199  La microarquitectura de los Apple M3, M3 Pro y...   

                                                enlace  
0    https://xataka.com/empresas-y-economia/asml-ti...  
1    https://xataka.com/ordenadores/estos-tres-pais...  
2    https://xataka.com/robotica-e-ia/llevamos-mese...  
3    https://xataka.com/empresas-y-economia/eeuu-su...  
4    https://xataka.com/empresas-y-economia/a-nvidi...  
..                                                 ...  
195  https://xataka.com/seguridad/alianza-30-paises...  
196  https://xataka.com/robotica-e-ia/se-llama-aita...  
197  https://xataka.com/aplicaciones/windows-10-se-...  
198  https://xataka.com/empresas-y-economia/samsung...  
199  https://xataka.com/componentes/microarquitectu...  

[200 rows x 2 columns]

... y para cada artículo extraemos el contenido y el enlace para poder acceder a la información completa.

In [89]:
quit()

Cerramos la sesión.

# Scrapy:

In [ ]:
pip install scrapy

In [ ]:
!scrapy startproject xataka

Iniciamos nuestro proyecto xataka, instalando scrapy en nuestro entorno de desarrollo. Xataka se trata de una web de tecnología, y en este caso nos centramos en sus noticias.

Accedemos a la carpeta del proyecto y creamos la carpeta spiders, donde guardaremos nuestra 'araña', que recogerá y mostrará la información de cada una de las páginas disponibles.

In [ ]:
cd ./xataka

In [8]:
%%writefile spiders/articles_spider.py

import pandas as pd
import scrapy
import logging
import datetime
import pytz

xatakas = pd.DataFrame(columns=['Titulo', 'Taxonomia', 'Extracto', 'URL'])

logging.getLogger('scrapy').propagate = False

class QuotesSpider(scrapy.Spider):
  name = "articles"
  start_urls = ["https://api.scrapingdog.com/scrape?api_key=65bb87fb6f171f73bf007799&url=https://www.xataka.com/tag/tecnologia"]
  custom_settings = {'LOG_LEVEL': logging.ERROR}
  pagina = 0
  fila = pagina * 10

  def parse(self, response):
    for article in response.css("article"):
      self.fila += 1
      xatakas.loc[self.fila, 'Titulo'] = article.css("h2.abstract-title a::text").get()
      xatakas.loc[self.fila, 'Taxonomia'] = article.css("a.abstract-taxonomy::text").get()
      xatakas.loc[self.fila, 'Extracto'] = article.css("div.abstract-excerpt p::text").get()
      xatakas.loc[self.fila, 'URL'] = article.css("a::attr(href)").get()
      self.fila += 1

    next_page = response.css("a.btn-next::attr(href)").get()
    if next_page is not None:
      self.pagina += 2
      yield response.follow(next_page, callback=self.parse, meta={'pagina': self.pagina})
    xatakas.to_csv('/content/xatakas.csv', index=False)

Overwriting spiders/articles_spider.py


Para cada artículo de las noticias mostramos el título, la taxonomía, el extracto y el enlace. Omitimos expresamente la información del log y guardamos el dataframe xatakas en un documento csv.

In [16]:
import pandas as pd
import datetime
import pytz

In [17]:
!scrapy crawl articles

fecha_hora = datetime.datetime.utcnow().astimezone(pytz.timezone('Europe/Madrid')).strftime('%Y-%m-%d %H:%M:%S')
print('Guardado en xatakas ('+fecha_hora+')')

Guardado en xatakas (2024-02-02 13:05:58)


In [18]:
xatakas = pd.read_csv('/content/xatakas.csv')

Recuperamos el documento csv como dataframe y mostramos su contenido.

In [19]:
xatakas

Titulo            Taxonomia  \
0    ASML tiene el mejor equipo de litografía del m...  Empresas y economía   
1    Estos son los tres países y las cinco empresas...          Ordenadores   
2    Llevamos meses esperando la gran jugada de App...        Robótica e IA   
3    EEUU no quiere perder la guerra de los chips. ...  Empresas y economía   
4    La IA ha disparado las previsiones de ventas d...  Empresas y economía   
..                                                 ...                  ...   
195  Una alianza de 30 países, entre ellos España, ...            Seguridad   
196  Se llama Aitana, es influencer y triunfa en In...        Robótica e IA   
197  Windows 10 se quedará sin soporte en dos años:...         Aplicaciones   
198  Samsung ha pisado el acelerador: esta es la te...  Empresas y economía   
199  La microarquitectura de los Apple M3, M3 Pro y...          Componentes   

                                              Extracto  \
0    Un equipo de litografía de ultravioleta extrem...   
1    La inversión económica habitualmente está alin...   
2                 Los misterios empiezan a resolverse.   
3    El Gobierno de EEUU está decidido a fortalecer...   
4    NVIDIA lidera el mercado de los chips para apl...   
..                                                 ...   
195  Se trata de una forma de cortar de raíz la fin...   
196  La IA generativa está evolucionando a una velo...   
197  Una organización teme que muchos de los ordena...   
198  Los planes de la filial de Samsung especializa...   
199  La familia de procesadores M3 de Apple ya está...   

                                                   URL  
0    https://www.xataka.com/empresas-y-economia/asm...  
1    https://www.xataka.com/ordenadores/estos-tres-...  
2    https://www.xataka.com/robotica-e-ia/llevamos-...  
3    https://www.xataka.com/empresas-y-economia/eeu...  
4    https://www.xataka.com/empresas-y-economia/a-n...  
..                                                 ...  
195  https://www.xataka.com/seguridad/alianza-30-pa...  
196  https://www.xataka.com/robotica-e-ia/se-llama-...  
197  https://www.xataka.com/aplicaciones/windows-10...  
198  https://www.xataka.com/empresas-y-economia/sam...  
199  https://www.xataka.com/componentes/microarquit...  

[200 rows x 4 columns]

In [20]:
quit()

Por último, hemos visto como extraer contenido de 3 sitios web distintos, utilizando tres herramientas diferentes: BeautifulSoup, Selenium y Scrapy, trabajando con sitios web tanto estáticos (quotes.toscrape) como dinámicos (wikipedia y xataka), documentando el proceso en los dos primeros casos. Como nota adicional, comentar que hemos utilizado la api de scrapingdog para enmascarar nuestro acceso al contenido de xataka y evitar así el bloqueo de la conexión.